In [1]:
import os
import pandas as pd
import numpy as np
import datasets
from datasets import load_dataset

In [2]:
import torch
from collections import Counter
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [3]:
curr_path = os.getcwd()
curr_path

'/home/anqizhao/Anqi/Bert Classifier/Bert_final code'

In [4]:
df = pd.read_csv('hdf_service_20230117_sentences.csv', sep='\t', encoding='utf_8_sig')
#df.to_csv('test.csv',sep='\t', index=False)

In [5]:
df_test = pd.read_csv('test.csv', sep='\t', encoding='utf_8_sig')

In [6]:
prediction_folder = 'prediction'
if not os.path.exists(prediction_folder):
    os.makedirs(prediction_folder)

In [7]:
data_path = 'hdf_service_20230117_sentences.csv'
dataset = load_dataset('csv', delimiter='\t', data_files={'data': [data_path]})
print(dataset)
columns = dataset['data'].column_names
print(columns)

DatasetDict({
    data: Dataset({
        features: ['service_id', 'all_doctor_text'],
        num_rows: 3065549
    })
})
['service_id', 'all_doctor_text']


In [13]:
#label_col = '信息支持1-病情描述/分析/诊断'
#label_col = '信息支持2-科普读物/其他病人经历'
#label_col = '信息支持4-治疗建议'
#label_col = '信息支持'
#label_col= '情感支持'
#parameter = '_622_5e-5'
label_col = '信息支持3-线上/门诊或住院流程/时间/费用'
parameter = '_622_1e-4'
model_path = os.path.join(curr_path, 'Classifier', label_col[:5]+parameter +'_model')
print(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
def preprocess_function(examples):
    return tokenizer(examples['all_doctor_text'], truncation=True)
tokenized_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2) 
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
pred_dataset = tokenized_dataset["data"]
predictions = trainer.predict(pred_dataset)
preds = predictions.predictions.argmax(-1)
df = pd.read_csv('hdf_service_20230117_sentences.csv', sep='\t', encoding='utf_8_sig')
df[label_col]=preds
df[label_col].value_counts()
df.to_csv(os.path.join(curr_path, prediction_folder, 'prediction_'+label_col[:5]+'.csv'), index=False)
df

/home/anqizhao/Anqi/Bert Classifier/Classifier/信息支持3_622_1e-4_model


Map:   0%|          | 0/3065549 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,service_id,all_doctor_text,信息支持3-线上/门诊或住院流程/时间/费用
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,1
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,0
...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0
3065545,8887571559,['您好',0
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,0
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],0


In [9]:
# df_test[label_col]=preds
# df_test[label_col].value_counts()
# #df_test.to_csv(os.path.join(curr_path, prediction_folder, 'prediction_'+label_col[:5]+'.csv'), index=False)
# df_test

In [5]:
df = pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/hdf_service_20230117_sentences.csv', sep='\t', encoding='utf_8_sig')

In [6]:
df1=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_信息支持1.csv')
df2=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_信息支持2.csv')
df3=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_信息支持3.csv')
df4=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_信息支持4.csv')
df_info=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_信息支持.csv')
df_emo=pd.read_csv('/home/anqizhao/Anqi/Bert Classifier/prediction/prediction_情感支持.csv')

In [7]:
df1['id'] = df1.index + 1
df1

,service_id,all_doctor_text,信息支持1-病情描述/分析/诊断,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,1,5
...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,3065545
3065545,8887571559,['您好',0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],1,3065548


In [9]:
df2['id'] = df2.index + 1
df2

,service_id,all_doctor_text,信息支持1-病情描述/分析/诊断,信息支持2-科普读物/其他病人经历,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,0,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,0,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,0,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,0,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,1,0,5
...,...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,0,3065545
3065545,8887571559,['您好',0,0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,0,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],1,0,3065548


In [10]:
df3['id'] = df3.index + 1
df3

,service_id,all_doctor_text,信息支持3-线上/门诊或住院流程/时间/费用,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',1,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,1,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,0,5
...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,3065545
3065545,8887571559,['您好',0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,0,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],0,3065548


In [11]:
df4['id'] = df4.index + 1
df4

,service_id,all_doctor_text,信息支持4-治疗建议,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',1,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',1,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,0,5
...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,3065545
3065545,8887571559,['您好',0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],0,3065548


In [12]:
df_info['id'] = df_info.index + 1
df_info

,service_id,all_doctor_text,信息支持,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',1,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',1,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',1,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,1,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,1,5
...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,3065545
3065545,8887571559,['您好',0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],1,3065548


In [13]:
df_emo['id'] = df_emo.index + 1
df_emo

,service_id,all_doctor_text,情感支持,id
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,1
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,2
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,3
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,4
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,0,5
...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,3065545
3065545,8887571559,['您好',0,3065546
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,0,3065547
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],0,3065548


In [14]:
merged_df = pd.merge(df2, df3, on =['service_id','all_doctor_text','id'],how='outer')
merged_df = pd.merge(merged_df, df4, on =['service_id','all_doctor_text','id'],how='outer')
merged_df = pd.merge(merged_df, df_info, on =['service_id','all_doctor_text','id'],how='outer')
merged_df = pd.merge(merged_df, df_emo, on =['service_id','all_doctor_text','id'],how='outer')
merged_df

,service_id,all_doctor_text,信息支持1-病情描述/分析/诊断,信息支持2-科普读物/其他病人经历,id,信息支持3-线上/门诊或住院流程/时间/费用,信息支持4-治疗建议,信息支持,情感支持
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,0,1,1,0,1,0
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,0,2,0,1,1,0
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,0,3,0,1,1,0
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,0,4,1,0,1,0
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,1,0,5,0,0,1,0
...,...,...,...,...,...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,0,3065545,0,0,0,0
3065545,8887571559,['您好',0,0,3065546,0,0,0,0
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,0,3065547,0,1,1,0
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],1,0,3065548,0,0,1,0


In [15]:
merged_df.to_csv('/home/anqizhao/Anqi/Bert Classifier/hdf_service_20230117_sentences_with labels.csv', sep='\t', index=False)